In [21]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


In [22]:
model_path="../../models/hf"
model_path="../../models/hf_fin"
#model_path="../../models/hf_fin_shuffle"
#model_path="../../models/hf_10000"
model_path="../../models/hf/0312wiki300m_grad_en_to_ja_fin"
model_path="../../models/hf/0313wiki300m_grad_en_to_ja_fin_65k_vocab"
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)


/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [23]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, max_new_tokens=200, repetition_penalty=1.5)

In [26]:
import time
text_list=["今日はいい",
"富士山は",
#"質問: 今日の天気は? 回答:",
"I have a",
"Functional polymers are"           
]

for text in text_list:
    perp=perplexity(model,tokenizer,text)
    s_time=time.time()
    res=pipe(text)[0]["generated_text"]
    consumed_time=time.time()-s_time
    print("-------")
    print("perplexity: ",perp)
    print("time: ", consumed_time)
    print("time/character: ", consumed_time/len(res))
    print("output: ",res)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
perplexity:  347.31317138671875
time:  1.490873098373413
time/character:  0.004372061872062795
output:  今日はいいよ」と、2019年3月5日に「あの日が来るなら...」というタイトルでCDデビュー。
 「君を想う気持ち」(作詞・作曲:つんく 編曲:渡辺拓也)
 「あなたがいるから」(編曲:高橋諭一&鈴木俊介)
 「恋してるのにかぎってたくなるように」(編曲:山田孝之 & 鈴木亮二)
 「愛しすぎて」(編曲:佐藤準弥)
 「ひとりじゃない」(編曲:井上鑑)
 「ずっとそばにいたい」(編曲:中村泰士)
 「好きですか?」(編曲:松浦晃久)
 「夢に向かっているとき」(編曲:清水信吾)
 「今度こそきっと」(編曲:吉田正志)
 「涙そうだから」(編曲:河野伸太郎)
 「私だけには言えない」(編曲:橋本和樹)
 「もう少しでも愛してる!」(編曲:田中公平)
 「もう一度逢いたくて」(


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
perplexity:  14341.2353515625
time:  1.487334966659546
time/character:  0.0038833811139935923
output:  富士山は、1970年(昭和45)に「富士登山」としてオープンした。

2016年には、「富士山の日」と題するイベントを開催し、同年8月には第3回東京箱根間往復大学駅伝競走大会を挙行したほか、翌々年に開催予定の第1回大会も開催されたが、いずれも中止となった。また、この年から毎年開催される箱根駅伝では、関東地区での開催が決定していることから、本レースへの参戦を見送るファンも多いという。なお、本大会には参加していないものの、優勝賞金や出場回数などの条件付きでの出走権を得ることを目標に掲げている。

コース・記録

歴代最多勝利数:1,000人/チーム内最高勝率.3% (2位/チーム内最低勝率.2%) ※史上最年長かつ日本人選手による連続得点記録は歴代最長である。

脚注

注釈

出典

関連項目 
 日本の陸上競技史 - 日本陸上競技選手権大会の記録一覧#日本


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
perplexity:  149.58096313476562
time:  1.4747555255889893
time/character:  0.00270101744613368
output:  I have a 2019–present partnership with the American rapper and producer, The New York Times.

Discography

Studio albums

Singles

References

External links
Official website of "The New Yorker" at Discogs (in English)
Review by David Bowie on YouTube

American hip hop musicians 『CLIMAX』(シー・モード、原題:Climax)は、アメリカ合衆国のテレビドラマ。シーズン4第3話「復讐の罠」(2016年5月8日)。日本ではWOWOWで2017年に放送されたが、『刑事コロンボ』シリーズとして放送されなかった(後述参照)。また、Netflixでも放送された他、Huluでは同年から放送されている。

あらすじ 
ロサンゼルス在住の警察官であるジョン・マッカーディと、その妻に先立たれた元恋人のエイミーがいることを知った彼は、FBI捜査官のマイケル・スナイダーと共に捜査に乗り出す。しかし、そこには、かつての恋人
-------
perplexity:  281.8184509277344
time:  1.4736251831054688
time/character:  0.0013619456405780672
output:  Functional polymers are used to produce a variety of proteins that can be found in the cellular membrane.

The protein encoded by this gene is expressed on chromosome 10, which has been shown as an example for mutation-binding a